In [1]:
# imports
import matplotlib.pyplot as plt
import matplotlib
import cv2
import os
import torch 
import numpy as np
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import time
import random
import sys

import torchvision.transforms as transforms
import torchvision.datasets as datasets

sys.path.append('../')

from tqdm import tqdm

from resnet import *
from main import *

In [2]:
import easydict 
args = easydict.EasyDict({ "batch-size": 256, 
                          "epochs": 80, 
                          "data": 0, 
                          'arch':'resnet18',
                          'lr':0.1,
                         'momentum':0.9,
                         'weight_decay':1e-4,
                         'start_epoch':0,
                         'gpu':0,
                          'print_freq':3000})

In [3]:
device = 'cuda'
model = resnet18(pretrained=False)
model.to(device)
criterion = nn.CrossEntropyLoss().to(device)
optimizer = torch.optim.SGD(model.parameters(), args.lr,
                            momentum=args.momentum,
                            weight_decay=args.weight_decay)

In [4]:
# Data loading code
data_dir = '../ILSVRC/Data/CLS-LOC/'
traindir = os.path.join(data_dir, 'train')
valdir = os.path.join(data_dir, 'val')

In [5]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                 std=[0.229, 0.224, 0.225])

train_dataset = datasets.ImageFolder(
    traindir,
    transforms.Compose([
        transforms.RandomResizedCrop(224),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        normalize,
    ]))
val_dataset = datasets.ImageFolder(valdir, transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        normalize,
    ]))

In [6]:
train_sampler = None
train_loader = torch.utils.data.DataLoader(
    train_dataset, batch_size=256, shuffle=(train_sampler is None),
    num_workers=8, pin_memory=True, sampler=train_sampler)

In [7]:
val_loader = torch.utils.data.DataLoader(
    val_dataset,
    batch_size=32, shuffle=False,
    num_workers=4, pin_memory=True)

In [8]:
model.to(device)

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (gelu): GELU()
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): 

In [9]:
best_acc1 = 0
acc1 = 0
train_loss = []
val_acc = []
for epoch in range(args.start_epoch, args.epochs):
    start_time = time.time()
    adjust_learning_rate(optimizer, epoch, args)

    # train for one epoch
    epoch_loss = train(train_loader, model, criterion, optimizer, epoch, args)

    # evaluate on validation set
    acc1 = validate(val_loader, model, criterion, args)  
    
    train_loss.append(epoch_loss)
    val_acc.append(acc1)
    print('************train_loss {} val_acc {}*************'.format(epoch_loss, acc1))
    
    # remember best acc@1 and save checkpoint
    is_best = acc1 > best_acc1
    best_acc1 = max(acc1, best_acc1)

#     if not args.multiprocessing_distributed or (args.multiprocessing_distributed
#             and args.rank % ngpus_per_node == 0):
    save_checkpoint({
        'epoch': epoch + 1,
        'arch': args.arch,
        'state_dict': model.state_dict(),
        'best_acc1': best_acc1,
        'optimizer' : optimizer.state_dict(),
    }, is_best)
    
    print(model.LT)
    print('epoch time', time.time() - start_time)

/home/beomgon/anaconda3/envs/pytorch_retina/lib/python3.7/site-packages/torch/nn/functional.py:718: UserWarning: Named tensors and all their associated APIs are an experimental feature and subject to change. Please do not use them for anything important until they are released as stable. (Triggered internally at  /pytorch/c10/core/TensorImpl.h:1156.)
  return torch.max_pool2d(input, kernel_size, stride, padding, dilation, ceil_mode)


Epoch: [0][   0/5005]	Time  2.832 ( 2.832)	Data  2.411 ( 2.411)	Loss 7.0142e+00 (7.0142e+00)	Acc@1   0.00 (  0.00)	Acc@5   0.39 (  0.39)
Epoch: [0][3000/5005]	Time  0.314 ( 0.634)	Data  0.000 ( 0.001)	Loss 4.5555e+00 (5.3605e+00)	Acc@1  15.23 (  7.19)	Acc@5  34.77 ( 18.86)
Test: [   0/1565]	Time  0.344 ( 0.344)	Loss 2.6602e+00 (2.6602e+00)	Acc@1  53.12 ( 53.12)	Acc@5  65.62 ( 65.62)
 * Acc@1 16.006 Acc@5 37.328
************train_loss None val_acc 16.006473541259766*************
Parameter containing:
tensor([[ 2.6945, -0.4133, -1.9093],
        [ 1.2610,  3.7624, -2.9204],
        [ 1.4330,  0.1944,  0.5236]], device='cuda:0', requires_grad=True)
epoch time 3213.737607240677
Epoch: [1][   0/5005]	Time  2.745 ( 2.745)	Data  2.294 ( 2.294)	Loss 3.7242e+00 (3.7242e+00)	Acc@1  23.83 ( 23.83)	Acc@5  48.05 ( 48.05)
Epoch: [1][3000/5005]	Time  0.312 ( 0.621)	Data  0.000 ( 0.001)	Loss 3.0290e+00 (3.5681e+00)	Acc@1  35.94 ( 26.98)	Acc@5  62.89 ( 50.97)
Test: [   0/1565]	Time  0.349 ( 0.349)	Loss

KeyboardInterrupt: 

In [ ]:
weight = torch.tensor([[ 1.6376, -0.2284,  0.0860],
        [ 0.1055,  1.9402, -0.0046],
        [-0.0257, -0.2146,  1.5572]], device='cuda:0', requires_grad=True).detach()

In [ ]:
torch.matmul(weight,  torch.inverse(weight))

In [ ]:
weight

In [ ]:
torch.inverse(weight)

In [ ]:
torch.inverse(model.LT.detach())

In [ ]:
path = '/home/Dataset/scl/patch_images/2021.01.15/LBC521-20210112(1)/LBC521-20210112(1)_1006.png'

In [ ]:
# img = cv2.imread(path)
# img = np.resize(img, (256,256,3))
# img = torch.tensor(img)/255.
# img.shape
img = torch.randn(2,256,256,3)

In [ ]:
weight = nn.Parameter(torch.eye(3, dtype=torch.float), requires_grad=True)
weight

In [ ]:
img[0][0:5]

In [ ]:
res = torch.matmul(img, weight)
# res[0]
res.shape

In [ ]:
res.mean().backward()

In [ ]:
weight.grad